In [ ]:
#This notebook was used to tune the hyperparameters for the Inception V3 Model

In [ ]:
import keras_tuner as kt
import tensorflow as tf
import numpy as np
import os
import random
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt

from tensorflow.keras.preprocessing.image import ImageDataGenerator


from sklearn.utils import class_weight



# -------------------------------------

tfk = tf.keras
tfkl = tf.keras.layers
print(tf.__version__)



# Random seed for reproducibility
seed = 42

random.seed(seed)
os.environ['PYTHONHASHSEED'] = str(seed)
np.random.seed(seed)
tf.random.set_seed(seed)
tf.compat.v1.set_random_seed(seed)



# Dataset folders 
dataset_dir = '/kaggle/input/dataset'
training_dir = os.path.join(dataset_dir, 'training')

height = 299
width = 299

In [ ]:
#augmented + thresholding, validation has only preprocessing
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.inception_v3 import preprocess_input


data_gen = ImageDataGenerator(validation_split=0.2,
                              fill_mode = 'constant',
                              rotation_range = 60,
                              horizontal_flip = True,
                              brightness_range = (0.8,1.2),
                              vertical_flip = True,
                              zoom_range = 0.1,
                              preprocessing_function = preprocess_input
                              )

train_gen = data_gen.flow_from_directory(directory = training_dir,
                                                 subset = 'training',
                                                 target_size=(height,width),
                                                 color_mode='rgb',
                                                 classes=None,
                                                 class_mode='categorical',
                                                 #batch_size=8,
                                                 shuffle=True,
                                                 seed=seed)

valid_gen = ImageDataGenerator(validation_split = 0.2,
                               preprocessing_function = preprocess_input).flow_from_directory(directory = training_dir,
                                                 subset = 'validation',
                                                 target_size=(height,width),
                                                 color_mode='rgb',
                                                 classes=None,
                                                 class_mode='categorical',
                                                 #batch_size=8,
                                                 shuffle=False,
                                                 seed=seed)
labels = np.unique(train_gen.classes)

# __Model__ fine-tuning

In [ ]:
input_shape = (299, 299, 3)

In [ ]:
model = tfk.models.load_model("../input/finetuning/SubmissionModel")

In [ ]:
def hypermodel(hp):
    model.get_layer('inception_v3').trainable = False
    hp_layers = hp.Choice('layers', values=[152, 216, 249])   
    for layer in model.get_layer('inception_v3').layers[hp_layers:]:    
        if not isinstance(layer, tfk.layers.BatchNormalization):
            layer.trainable = True

    hp_learningrate = hp.Choice('learning_rate', values=[2.5e-3, 1e-3, 1e-4])
    hp_weightdecay = hp.Choice('weight_decay', values=[1e-5, 1e-6])
    
    #opt1 = tfk.optimizers.SGD(learning_rate=hp_learningrate, momentum=0.9, decay=hp_weightdecay)
    opt2 = tfk.optimizers.Adam(learning_rate=hp_learningrate, decay=hp_weightdecay)
    #opt3 = tfk.optimizers.RMSprop(learning_rate=hp_learningrate)
    
    hp_loss = hp.Choice('loss', values=['kl_divergence'])
    
    model.compile(optimizer=opt2, loss=hp_loss, metrics=["accuracy"])
    
    return model

In [ ]:
%cd /kaggle/working

In [ ]:
tuner = kt.RandomSearch(hypermodel,
                        objective='val_accuracy',
                        executions_per_trial=1, 
                        directory='tuning',
                        max_trials = 5,
                        project_name='hyperparam-tune')
print('ok')

In [ ]:
tuner.search_space_summary()

In [ ]:
tuner.search(x = train_gen,   
            epochs = 5,
            validation_data = valid_gen)

In [ ]:
models = tuner.get_best_models(num_models=2)

In [ ]:
tuner.results_summary()